# Использование предобученных моделей  (Transfer Learning)

Задание: загрузить датасет и использовать перенос обучения для решения задачи классификации.

## Порядок выполнения

1. Загрузить датасет ;
1. Подготовить transforms, DataSet и DataLoader;
1. Выбрать одну из моделей
Список предобученных моделей в Pytorch.(https://pytorch.org/vision/stable/models.html#classification)
1. Использовать на этой модели прием выделения признаков;
1. Использовать на этой модели прием дообучения (fine-tune);
1. Оценить результаты лучшей модели на тестовой выборке.
Написать выводы по итогам работы.
```




In [1]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import matplotlib.pyplot as plt
import PIL
import random


%matplotlib inline

<ipython-input-1-52a54a514ba7>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qo hymenoptera_data.zip -d data/

In [ ]:
os.listdir('/content/data')

['hymenoptera_data']

## Создание Dataset и DataLoader

По аналогии с прошлыми заданиями требуется создать transforms, которые передаются в создаваемый Dataset и из датасета  создаете DataLoaders.

Можно использовать:

- v2.ToImage()
- v2.Resize() или v2.RandomResizedCrop() - размер изображения после кадрирования, должен быть равен размеру ожидаемому на ходе предобученной модели.
- v2.RandomRotation()
- v2.RandomHorizontalFlip()
- v2.ToDtype()

[Описания в документации](https://pytorch.org/vision/stable/transforms.html#v2-api-reference-recommended)

In [3]:
# Аугментация обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/hymenoptera_data/hymenoptera_data'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# создание dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Извлечение признаков

## Прием извлечения признаков

Этот прием может использоваться как сам по себе, так и быть предварительным этапом для дообучения.

Он заключается в том, что заменяется полносвязная часть модели (head/голова) на свою с учетом размерностей выходных данных из сверточной части и количеством классов в текущей задаче. Перед обучением требуется "заморозить" параметры сверточных слоев.



[Список](https://pytorch.org/vision/stable/models.html#classification)

1.   Новый пункт
2.   Новый пункт

предобученных моделей в Pytorch.

### Загрузка модели
У модели можно использовать метод .parameters(), он возвращает итерируемый объект с параметрами вашей модели, (можно их перебрать и отключить необходимость расчета градиентов).

In [4]:
num_classes = 2 # Количество классов в вашей задаче
#model_conv = torchvision.models.resnet101(pretrained=True)
model_conv = torchvision.models.resnet18(pretrained=True) # Выбирите 2-3 модели из списка и дообучите их (pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Получение количества входных признаков в полносвязном слое (fc) предобученной модели.
# Параметры вновь созданных моделей по умолчанию имеют requires_grad=True
num_ftrs = model_conv.fc.in_features

# Замена полносвязного слоя (fc) предобученной модели.
model_conv.fc = nn.Linear(num_ftrs, num_classes)

model_conv = model_conv.to(device)


# Создание функции потерь
loss_fn = nn.CrossEntropyLoss()

# Создание оптимизатора
# Обратите внимание, что оптимизируются только параметры последнего слоя, в отличие от предыдущего.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 188MB/s]


В этой работе также рассмотрим применение планировщика для изменения скорости обучения. Ранее скорость обучения была константой, теперь же в процессе обучения каждые n эпох будем ее снижать.

In [5]:
from torch.optim import lr_scheduler

# Создание экземпляра StepLR
# Уменьшение LR в 0,1 раза каждые 7 эпох
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

### ### Обучение и тестирование

Несколько эпох обучите модель в таком состоянии. Для обучения используйте цикл с эпохами и перебором dataloader, но к нему в цикл эпох требуется добавить шаг планировщика scheduler.step().

Функция обучения с переключением режимов моделей (model.train(), model.eval()), так как теперь в них может быть пакетная нормализация и используйте контекст torch.no_grad() при проверке модели.

In [6]:
def train_model(model, loss_fn, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        ## Каждая эпоха имеет обучающую и тестовую фазу
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # # Установка модели в режим обучения
            else:
                model.eval()   # Установка модели в режим оценки

            running_loss = 0.0
            running_corrects = 0

            # Итерация по данным
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуление градиентов параметров
                optimizer.zero_grad()

                # forward
                # Обучение только для режима  'train'
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = loss_fn(outputs, labels)

                    # backward + запуск только для обучающей фазы
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Сбор статистик
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # лучшая модель
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # сохранение модели
    model.load_state_dict(best_model_wts)
    return model

### Обучение и оценка

In [7]:
model_ft = train_model(model_conv, loss_fn, optimizer_conv, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.5573 Acc: 0.6885
val Loss: 0.3860 Acc: 0.8105

Epoch 1/9
----------
train Loss: 0.5211 Acc: 0.7172
val Loss: 0.2256 Acc: 0.9477

Epoch 2/9
----------
train Loss: 0.4539 Acc: 0.7705
val Loss: 0.3421 Acc: 0.8431

Epoch 3/9
----------
train Loss: 0.5362 Acc: 0.7418
val Loss: 0.1936 Acc: 0.9477

Epoch 4/9
----------
train Loss: 0.5505 Acc: 0.7377
val Loss: 0.3071 Acc: 0.8758

Epoch 5/9
----------
train Loss: 0.4539 Acc: 0.8033
val Loss: 0.2576 Acc: 0.9085

Epoch 6/9
----------
train Loss: 0.5266 Acc: 0.7500
val Loss: 0.1585 Acc: 0.9346

Epoch 7/9
----------
train Loss: 0.3740 Acc: 0.8484
val Loss: 0.1576 Acc: 0.9608

Epoch 8/9
----------
train Loss: 0.3461 Acc: 0.8443
val Loss: 0.1678 Acc: 0.9412

Epoch 9/9
----------
train Loss: 0.3503 Acc: 0.8566
val Loss: 0.1531 Acc: 0.9412

Training complete in 1m 41s
Best val Acc: 0.960784


## Анализ данных


Проанализировать данные и сделать выводы о возможных проблемах.



#  Задание

Решить  задачу классификации используя перенос обучения.

Определить задачу по классификации изображений основываясь на  теме исследований. (придумать абстрактную задачу).

В рамках задачи выбрать количество классов объектов. На каждый класс подобрать минимум по 30 изображений и распределить их на обучающую (train) и проверочную выборку (val).

Выбрать 2-3 уже обученные модели, дообучить их на своих данных и оценить результат.
# Порядок выполнения
1. Cоздать датасет ;
2. Подготовить transforms, DataSet и DataLoader;
3. Выбрать одну из моделей Список предобученных моделей в Pytorch.(https://pytorch.org/vision/stable/models.html#classification)
4. Использовать на этой модели прием выделения признаков;
5. Использовать на этой модели прием дообучения (fine-tune);
6. Оценить результаты лучшей модели на тестовой выборке.




## Создание датасета

В этом задании нужно собрать собственный датасет из изображений. В нем должно быть минимум 30 изображений для каждого класса. Количество классов не менее 2.

Далее в этом разделе приведен пример кода, который помогает скачать изображения по запросу на гугл диск.

In [8]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [9]:
# Установка пакета для работы с API поисковика DuckDuckGo

!pip install -U duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.1 MB/s eta 0:00:00


In [22]:
import os
import random
import requests
from duckduckgo_search import DDGS
from tqdm import tqdm

base_dir = "/content/drive/My Drive/data"

train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

for split in [train_dir, val_dir]:
    for category in ["dog", "cat"]:
        os.makedirs(os.path.join(split, category), exist_ok=True)

print("Директории созданы!")

def download_images(query, category):
    with DDGS() as ddgs:
        images = list(ddgs.images(query, region="wt-wt", size="Medium", type_image="photo", max_results=30))

    random.shuffle(images)
    train_images = images[:24]
    val_images = images[24:]

    for split, img_list in zip([train_dir, val_dir], [train_images, val_images]):
        for i, img in enumerate(tqdm(img_list, desc=f"Downloading {category} in {split}")):
            img_url = img["image"]
            ext = img_url.split(".")[-1].split("?")[0]
            img_path = os.path.join(split, category, f"{i}.{ext}")

            try:
                response = requests.get(img_url, timeout=10)
                if response.status_code == 200:
                    with open(img_path, "wb") as f:
                        f.write(response.content)
            except Exception as e:
                print(f"Ошибка при скачивании {img_url}: {e}")

download_images("dog", "dog")
download_images("cat", "cat")

print("Загрузка завершена!")

Директории созданы!


Загрузка завершена!


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!mkdir "/content/drive/My Drive/data"

mkdir: cannot create directory ‘/content/drive/My Drive/data’: File exists


In [25]:
#  содержимое файла со ссылками
!cat cat.txt

https://hips.hearstapps.com/hmg-prod/images/close-up-of-cat-sitting-on-wood-turkish-angora-cat-royalty-free-image-1658453231.jpg
https://www.catster.com/wp-content/uploads/2023/11/tuxedo-ragamuffin-cat-on-the-table_Kill_Baal_Shutterstock.jpg
http://4.bp.blogspot.com/-iZHutM3STzc/T-xa8r1FsQI/AAAAAAAABEE/0HHeGcJ7bhc/s1600/ragdoll-cat4.jpeg
https://www.fearfreehappyhomes.com/wp-content/uploads/2020/08/shutterstock_75059266.jpg
https://c.pxhere.com/photos/d4/9c/pet_cat_thinking-907545.jpg!s
https://c4.wallpaperflare.com/wallpaper/447/224/54/baby-cat-cats-cute-wallpaper-preview.jpg
https://www.shutterstock.com/image-photo/curious-calico-cat-walking-outside-260nw-1765073558.jpg
https://png.pngtree.com/background/20230612/original/pngtree-orange-cat-in-autumn-leaves-with-blue-eyes-picture-image_3173627.jpg
https://www.rover.com/blog/wp-content/uploads/2020/03/cat-620030_1920-960x540.jpg
https://www.thesprucepets.com/thmb/ZguTGsZgHTGK4One8sKHZJOVNaw=/2119x0/filters:no_upscale():strip_icc()/Get

In [26]:
# утилита wget построчно читает файл dog.txt и скачивает по URL файлы в папку,
# указанную после флага -P. --random-wait добавляет случайные интервалы между запросами,
# чтобы снизить вероятность блокировки

!wget -i dog.txt --random-wait -P "/content/drive/My Drive/data/cat"

--2025-03-21 09:53:14--  http://www.publicdomainpictures.net/pictures/40000/velka/rough-collie-dog-1361732766Q2h.jpg
Resolving www.publicdomainpictures.net (www.publicdomainpictures.net)... 104.20.122.60, 104.20.123.60, 172.67.1.236, ...
Connecting to www.publicdomainpictures.net (www.publicdomainpictures.net)|104.20.122.60|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.publicdomainpictures.net/pictures/40000/velka/rough-collie-dog-1361732766Q2h.jpg [following]
--2025-03-21 09:53:14--  https://www.publicdomainpictures.net/pictures/40000/velka/rough-collie-dog-1361732766Q2h.jpg
Connecting to www.publicdomainpictures.net (www.publicdomainpictures.net)|104.20.122.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 467799 (457K) [image/jpeg]
Saving to: ‘/content/drive/My Drive/data/cat/rough-collie-dog-1361732766Q2h.jpg’

rough-collie-dog-13 100%[===================>] 456.83K  --.-KB/s    in 0.02s   

2025-03-2

#
В результате работы по этому разделу должен получиться датасет.
Проверьте что все скачанные изображения открываются и удалите поврежденные файлы. Изображения необходимо разделить в папках на train и test.

# Fine-tuning

In [27]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
import time
import copy
import matplotlib.pyplot as plt

In [28]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [30]:
data_dir = "/content/drive/My Drive/data"

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=2) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
model_ft = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [32]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print(f"Лучшее валидационное качество: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

In [33]:
model_ft = train_model(model_ft, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10
----------
train Loss: 0.7848 Acc: 0.5135
val Loss: 0.4761 Acc: 0.6364
Epoch 2/10
----------
train Loss: 0.5335 Acc: 0.7297
val Loss: 0.2832 Acc: 1.0000
Epoch 3/10
----------
train Loss: 0.2280 Acc: 0.9730
val Loss: 0.2041 Acc: 1.0000
Epoch 4/10
----------
train Loss: 0.2577 Acc: 0.8378
val Loss: 0.2049 Acc: 0.9091
Epoch 5/10
----------
train Loss: 0.0781 Acc: 1.0000
val Loss: 0.2175 Acc: 0.9091
Epoch 6/10
----------
train Loss: 0.0911 Acc: 1.0000
val Loss: 0.1761 Acc: 0.9091
Epoch 7/10
----------
train Loss: 0.1227 Acc: 1.0000
val Loss: 0.1577 Acc: 0.9091
Epoch 8/10
----------
train Loss: 0.1108 Acc: 0.9459
val Loss: 0.1862 Acc: 0.9091
Epoch 9/10
----------
train Loss: 0.0526 Acc: 1.0000
val Loss: 0.1921 Acc: 0.9091
Epoch 10/10
----------
train Loss: 0.0582 Acc: 1.0000
val Loss: 0.1853 Acc: 0.9091
Лучшее валидационное качество: 1.0000


In [34]:
def evaluate_model(model):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / dataset_sizes['val']
    print(f'Accuracy on validation set: {acc:.4f}')

evaluate_model(model_ft)

Accuracy on validation set: 1.0000


# Выделения признаков

In [35]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import copy

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = "/content/drive/My Drive/data"

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=2) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_ft = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print(f"Лучшее валидационное качество: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model_ft, criterion, optimizer, scheduler, num_epochs=10)

def evaluate_model(model):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / dataset_sizes['val']
    print(f'Accuracy on validation set: {acc:.4f}')

evaluate_model(model_ft)

Epoch 1/10
----------
train Loss: 0.7944 Acc: 0.5676
val Loss: 0.5966 Acc: 0.7273
Epoch 2/10
----------
train Loss: 0.6523 Acc: 0.5405
val Loss: 0.4318 Acc: 0.9091
Epoch 3/10
----------
train Loss: 0.4596 Acc: 0.7838
val Loss: 0.4144 Acc: 0.8182
Epoch 4/10
----------
train Loss: 0.2890 Acc: 0.9189
val Loss: 0.2858 Acc: 0.9091
Epoch 5/10
----------
train Loss: 0.2116 Acc: 0.9730
val Loss: 0.2543 Acc: 0.9091
Epoch 6/10
----------
train Loss: 0.1987 Acc: 0.9459
val Loss: 0.2409 Acc: 0.9091
Epoch 7/10
----------
train Loss: 0.1423 Acc: 1.0000
val Loss: 0.2498 Acc: 0.9091
Epoch 8/10
----------
train Loss: 0.1287 Acc: 1.0000
val Loss: 0.2591 Acc: 0.9091
Epoch 9/10
----------
train Loss: 0.1272 Acc: 1.0000
val Loss: 0.2551 Acc: 0.9091
Epoch 10/10
----------
train Loss: 0.1935 Acc: 0.9189
val Loss: 0.2808 Acc: 0.9091
Лучшее валидационное качество: 0.9091
Accuracy on validation set: 0.9091
